In [2]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

In [3]:
DATA_DIR = "data/stanford_dogs/Images"
OUTPUT_DIR = "data/Processed"
IMG_SIZE = (224, 224)

In [4]:
def load_dataset(data_dir):
    all_paths = glob(os.path.join(data_dir, "*", "*.jpg"))
    all_labels = [os.path.basename(os.path.dirname(p)) for p in all_paths]
    return all_paths, all_labels

# image_paths, labels = load_dataset(DATA_DIR)
# print(f"Found {len(image_paths)} images")

In [5]:
def resize_image(image, target_size=IMG_SIZE):
    """Resize the image to the target size."""
    return cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

In [6]:
def normalize_image(image):
    """Normalize image pixel values to the range [0.0, 1.0]."""
    return image.astype(np.float32) / 255.0

In [7]:

def preprocess_image(path,
                     target_size=IMG_SIZE,
                     to_rgb=True,
                     normalize=True):
    """
    Load and preprocess an image from a given path:
    1) Read image file
    2) Convert BGR to RGB (if enabled)
    3) Resize to target size
    4) Normalize pixel values (if enabled)
    Returns the processed image or None if loading fails.
    """
    img = cv2.imread(path)
    if img is None:
        return None

    if to_rgb:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img = resize_image(img, target_size)

    if normalize:
        img = normalize_image(img)

    return img

In [8]:
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# for path, label in tqdm(zip(image_paths, labels), total=len(image_paths)):
#     image = cv2.imread(path)
#     if image is None:
#         continue
    
#      image = resize_image(image)
#      image = normalize_image(image)
#      image = preprocess_image(image)
    
#     save_path = os.path.join(OUTPUT_DIR, label)
#     os.makedirs(save_path, exist_ok=True)
#     filename = os.path.basename(path)
    
#     save_img = (np.clip(image, 0, 1) * 255).astype(np.uint8)
#     cv2.imwrite(os.path.join(save_path, filename), save_img)
if __name__ == "__main__":
    image_paths, labels = load_dataset(DATA_DIR)
    print(f"Found {len(image_paths)} images")

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    for path, label in tqdm(zip(image_paths, labels), total=len(image_paths),
                            desc="Preprocessing"):
        img = preprocess_image(path)
        if img is None:
            print(f"[READ FAIL] {path}")  # Warn if the image could not be read
            continue

        # Convert back to 0–255 and RGB→BGR before saving
        save_img = (img * 255).astype(np.uint8)
        save_img = cv2.cvtColor(save_img, cv2.COLOR_RGB2BGR)

        save_dir = os.path.join(OUTPUT_DIR, label)
        os.makedirs(save_dir, exist_ok=True)

        cv2.imwrite(os.path.join(save_dir, os.path.basename(path)), save_img)

Found 20580 images


Preprocessing:   0%|          | 0/20580 [00:00<?, ?it/s]